In [40]:
import os
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")
os.chdir("/workspace")
print(f"Working directory changed to: {os.getcwd()}")
from datasets import load_dataset, load_from_disk
import verifiers as vf
import json
import gc
import torch
import importlib
from verifiers.envs import bfcl_inthinking_env
import verifiers.rubrics.bfcl_inthinking_rubric

gc.collect()
torch.cuda.empty_cache()

Current working directory: /workspace
Working directory changed to: /workspace


# Loading Env

In [2]:
env = bfcl_inthinking_env.BfclITEnv(
    dataset="bfcl",
    tools=[],
    max_num_turns=1,
    max_steps_per_turn=10,
    curriculum_learning=True,
    use_latest_trl=False,
)

2025-04-20 14:31:44.098 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:281 - Initializing BfclITEnv (Single Tool Call Flow)
2025-04-20 14:31:44.098 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:315 - Initializing Scoring Rubric
2025-04-20 14:31:44.099 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:317 - Initializing LLM Parser


In [3]:
ds_train = env.get_dataset(max_num_turns=1)
ds_eval = env.get_eval_dataset(max_num_turns=1, max_turn_only=True)
example=ds_train[0]
ds_train

2025-04-20 14:31:44.183 | INFO     | verifiers.envs.bfcl_inthinking_env:get_dataset:324 - Preprocessing dataset bfcl for train split...
Parameter 'function'=<function preprocess_bfcl_dataset.<locals>.<lambda> at 0x7f1b146dc4a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

2025-04-20 14:31:44.917 | INFO     | verifiers.envs.bfcl_inthinking_env:get_dataset:335 - Filtering train dataset where num_turns <= 1.


...

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

2025-04-20 14:31:45.637 | INFO     | verifiers.envs.bfcl_inthinking_env:get_eval_dataset:361 - Filtering eval dataset where num_turns <= 1.


Filter:   0%|          | 0/366 [00:00<?, ? examples/s]

2025-04-20 14:31:45.653 | INFO     | verifiers.envs.bfcl_inthinking_env:get_eval_dataset:371 - Filtering eval dataset to keep only the max turn entry per ID.


Dataset({
    features: ['id', 'question', 'initial_config', 'path', 'involved_classes', 'num_total_turns_in_task', 'answer', 'num_turns', 'prompt', 'ground_truth_bank', '__index_level_0__'],
    num_rows: 100
})

In [4]:
rubric = env.get_rubric()
training_args = vf.get_default_grpo_config(run_name="debug", num_gpus=2)
training_args.num_train_epochs = 1
training_args.num_generations = 2
training_args.max_completion_length = 2048
training_args.report_to = "none"
training_args.per_device_train_batch_size = 2
training_args.gradient_accumulation_steps = 4
training_args.num_iterations = 2
training_args.beta = 0.001
training_args.max_grad_norm = 0.2
training_args.eval_strategy = "steps"
training_args.eval_on_start = False
training_args.eval_steps = 50
training_args.save_strategy = "steps"
training_args.save_steps = 100
training_args.per_device_eval_batch_size = 2
training_args.eval_accumulation_steps = 1
training_args.data_seed = 42
training_args.sync_ref_model = True
training_args.ref_model_mixup_alpha = 1.0
training_args.ref_model_sync_steps = 100

In [5]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
#model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model, tokenizer = vf.get_model_and_tokenizer(model_name)

trainer = vf.GRPOEnvTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=rubric,
    env=env,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    debug_generate=True,
    debug_rewards=True,
    model_name=model_name,
    run_name="debug_baseline",
    use_dr_grpo=False,
    test_hypothesis_clip_advantage=False,
    apply_overlong_filtering=True,
    print_sample_completions=True,
)

Using Liger kernel


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Applied Liger kernels to Qwen2
INFO 04-20 14:31:47 __init__.py:207] Automatically detected platform cuda.
INFO 04-20 14:31:55 config.py:549] This model supports multiple tasks: {'generate', 'classify', 'score', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 04-20 14:31:55 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-1.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:1, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forw

[W420 14:31:56.759409264 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 04-20 14:31:56 weight_utils.py:254] Using model weights format ['*.safetensors']
INFO 04-20 14:31:56 weight_utils.py:304] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-20 14:31:57 model_runner.py:1115] Loading model weights took 2.8833 GB
INFO 04-20 14:31:58 worker.py:267] Memory profiling takes 1.12 seconds
INFO 04-20 14:31:58 worker.py:267] the current vLLM instance can use total_gpu_memory (23.55GiB) x gpu_memory_utilization (0.70) = 16.48GiB
INFO 04-20 14:31:58 worker.py:267] model weights take 2.88GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 11.50GiB.
INFO 04-20 14:31:59 executor_base.py:111] # cuda blocks: 26912, # CPU blocks: 9362
INFO 04-20 14:31:59 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 13.14x
INFO 04-20 14:32:02 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:16<00:00,  2.14it/s]

INFO 04-20 14:32:18 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.22 GiB
INFO 04-20 14:32:18 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 21.27 seconds



Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
2025-04-20 14:32:20.071 | INFO     | verifiers.trainers.grpo_env_trainer:__init__:148 - Trainer sampling params set: stop=['</tool>', '<TASK_FINISHED>', '<TASK_ERROR>'], include_stop=True, ignore_eos=False


In [7]:
#trainer.train()

# Evaluating Outputs

In [31]:
print(example["prompt"][0]['content'])

You are an expert assistant that reasons step-by-step and uses tools to answer user questions.

# Available tools:
{"name": "add_contact", "description": "Add a contact to the workspace.", "parameters": {"type": "dict", "properties": {"user_name": {"type": "string", "description": "User name of contact to be added."}}, "required": ["user_name"]}, "response": {"type": "dict", "properties": {"added_status": {"type": "boolean", "description": "True if the contact was added successfully, False otherwise."}, "user_id": {"type": "string", "description": "User ID of the added contact."}, "message": {"type": "string", "description": "A message describing the result of the addition attempt."}}}}
{"name": "delete_message", "description": "Delete the latest message sent to a receiver.", "parameters": {"type": "dict", "properties": {"receiver_id": {"type": "string", "description": "User ID of the user to send the message to."}, "message_id": {"type": "integer", "description": "ID of the message to

In [63]:

importlib.reload(verifiers.rubrics.bfcl_inthinking_rubric)
importlib.reload(bfcl_inthinking_env)
env = bfcl_inthinking_env.BfclITEnv(
    dataset="bfcl",
    tools=[],
    max_num_turns=1,
    max_steps_per_turn=10,
    curriculum_learning=True,
    use_latest_trl=False,
)
rubric = env.get_rubric()
reward_func=rubric[0]
trainer.reward_funcs=rubric
trainer.env=env
trainer.env.tokenizer=trainer.tokenizer
ds_train = env.get_dataset(max_num_turns=1)
example=ds_train[0]
output = trainer.env.generate(
    prompts=[example["prompt"]],
    llm=trainer.llm,
    sampling_params=trainer.sampling_params,
    dataset_rows=[example],
    debug=True,  # True for detailed output
)

2025-04-20 15:42:22.965 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:281 - Initializing BfclITEnv (Single Tool Call Flow)
2025-04-20 15:42:22.966 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:315 - Initializing Scoring Rubric
2025-04-20 15:42:22.966 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:317 - Initializing LLM Parser
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
2025-04-20 15:42:22.966 | INFO     | verifiers.envs.bfcl_inthinking_env:get_dataset:324 - Preprocessing dataset bfcl for train split...


Map:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

2025-04-20 15:42:23.571 | INFO     | verifiers.envs.bfcl_inthinking_env:get_dataset:335 - Filtering train dataset where num_turns <= 1.


Filter:   0%|          | 0/379 [00:00<?, ? examples/s]

2025-04-20 15:42:23.587 | INFO     | verifiers.envs.bfcl_inthinking_env:generate:1206 - Using sampling params: stop=['</tool>', '<TASK_ERROR>', '<TASK_FINISHED>'], include_stop=True, max_tokens=2048


Initial User Prompt (State 0): You are an expert assistant that reasons step-by-step and uses tools to answer user questions.

# Available tools:
{"name": "add_contact", "description": "Add a contact to the workspace.", "parameters": {"type": "dict", "properties": {"user_name": {"type": "string", "description": "User name of contact to be added."}}, "required": ["user_name"]}, "response": {"type": "dict", "properties": {"added_status": {"type": "boolean", "description": "True if the contact was added successfully, False other...
Number of Rollouts: 1

--- Starting Generation Step 1 ---
---> Calling LLM for 1 states (add_generation_prompt=True)


2025-04-20 15:42:24.151 | DEBUG    | verifiers.envs.bfcl_inthinking_env:generate:1290 - --- End of Step Status ---
2025-04-20 15:42:24.152 | DEBUG    | verifiers.envs.bfcl_inthinking_env:generate:1292 - State 0 (ID: multi_turn_base_51): Completed=False, Error='None'
2025-04-20 15:42:24.152 | DEBUG    | verifiers.envs.bfcl_inthinking_env:generate:1293 - --------------------------
2025-04-20 15:42:24.152 | DEBUG    | verifiers.envs.bfcl_inthinking_env:step:654 - State 0 checking for continuation. Last message ends with: ', 'remainingUnlockedDoors': 0}"</tool_result> Ok, '
2025-04-20 15:42:24.152 | DEBUG    | verifiers.envs.bfcl_inthinking_env:step:722 - Input messages for continuation call (State 0):
[
  {
    "content": "You are an expert assistant that reasons step-by-step and uses tools to answer user questions.\n\n# Available tools:\n{\"name\": \"add_contact\", \"description\": \"Add a contact to the workspace.\", \"parameters\": {\"type\": \"dict\", \"properties\": {\"user_name\": {

--- Processing 1 LLM responses ---
State 0 LLM Output Segment: '**Think Step-by-Step:** The user needs to ensure that all car doors are locked. We need to first determine which doors can be locked and then confirm their status.

**Identify Tool Need:** We need to check the status of the doors and lock them as needed.

**Execute Tool Call (If Needed):**
```
<tool>{"name": "lockDoors", "args": {"unlock": false, "door": ["driver", "passenger", "rear_left", "rear_right"]}}</tool>'
State 0: Detected </tool>
Attempting Tool Call: lockDoors({'unlock': False, 'door': ('driver', 'passenger', 'rear_left', 'rear_right')})
Found method lockDoors in class VehicleControlAPI
------------------------------ Step End (Processed 1 LLM responses) ------------------------------
--- Finished Generation Step 1 (All Completed: False) ---

--- Starting Generation Step 2 ---
---> Calling LLM for 1 states (continue_final_message=True)
--- Processing 1 LLM responses ---
State 0 LLM Output Segment: '4 doors are cu

In [66]:
output['states']

[{'messages': [{'content': 'You are an expert assistant that reasons step-by-step and uses tools to answer user questions.\n\n# Available tools:\n{"name": "add_contact", "description": "Add a contact to the workspace.", "parameters": {"type": "dict", "properties": {"user_name": {"type": "string", "description": "User name of contact to be added."}}, "required": ["user_name"]}, "response": {"type": "dict", "properties": {"added_status": {"type": "boolean", "description": "True if the contact was added successfully, False otherwise."}, "user_id": {"type": "string", "description": "User ID of the added contact."}, "message": {"type": "string", "description": "A message describing the result of the addition attempt."}}}}\n{"name": "delete_message", "description": "Delete the latest message sent to a receiver.", "parameters": {"type": "dict", "properties": {"receiver_id": {"type": "string", "description": "User ID of the user to send the message to."}, "message_id": {"type": "integer", "des

In [64]:
print(output['messages'][0][0]['content'])

**Think Step-by-Step:** The user needs to ensure that all car doors are locked. We need to first determine which doors can be locked and then confirm their status.

**Identify Tool Need:** We need to check the status of the doors and lock them as needed.

**Execute Tool Call (If Needed):**
```
<tool>{"name": "lockDoors", "args": {"unlock": false, "door": ["driver", "passenger", "rear_left", "rear_right"]}}</tool><tool_result>"Function Call {'name': 'lockDoors', 'args': {'unlock': False, 'door': ('driver', 'passenger', 'rear_left', 'rear_right')}} Succeeded. Result: {'lockStatus': 'locked', 'remainingUnlockedDoors': 0}"</tool_result> Ok, 4 doors are currently locked and there are no unlocked ones left.
</think><|im_end|>


In [65]:
reward_func(output['messages'], output['states'], debug=True)

2025-04-20 15:42:24.736 | INFO     | verifiers.rubrics.bfcl_inthinking_rubric:unified_reward_func:390 - Calculating combined rewards...
2025-04-20 15:42:24.736 | DEBUG    | verifiers.rubrics.bfcl_inthinking_rubric:unified_success_reward_func:123 - 
--- Unified Reward Check (State 0) ---
2025-04-20 15:42:24.736 | DEBUG    | verifiers.rubrics.bfcl_inthinking_rubric:unified_success_reward_func:139 - Comparing state for class: MessageAPI
2025-04-20 15:42:24.737 | DEBUG    | verifiers.rubrics.bfcl_inthinking_rubric:unified_success_reward_func:139 - Comparing state for class: VehicleControlAPI
2025-04-20 15:42:24.737 | DEBUG    | verifiers.rubrics.bfcl_inthinking_rubric:unified_success_reward_func:144 - State mismatch found for class: VehicleControlAPI
2025-04-20 15:42:24.737 | DEBUG    | verifiers.rubrics.bfcl_inthinking_rubric:unified_success_reward_func:176 - Function calls match GT: True
2025-04-20 15:42:24.737 | DEBUG    | verifiers.rubrics.bfcl_inthinking_rubric:unified_success_reward_

[0.25]

In [73]:
vars(output['states'][0]['environment']['VehicleControlAPI'])

{'_api_description': 'This tool belongs to the vehicle control system, which allows users to control various aspects of the car such as engine, doors, climate control, lights, and more.',
 '_random': <random.Random at 0x555cd6e4b2c0>,
 'fuelLevel': 15.5,
 'batteryVoltage': 12.8,
 'engine_state': 'stopped',
 'remainingUnlockedDoors': 0,
 'doorStatus': {'driver': 'locked',
  'passenger': 'locked',
  'rear_left': 'locked',
  'rear_right': 'locked'},
 'acTemperature': 22.0,
 'fanSpeed': 60,
 'acMode': 'auto',
 'humidityLevel': 45.0,
 'headLightStatus': 'off',
 'parkingBrakeStatus': 'released',
 '_parkingBrakeForce': 0.0,
 '_slopeAngle': 0.0,
 'brakePedalStatus': 'released',
 '_brakePedalForce': 0.0,
 'distanceToNextVehicle': 100.0,
 'cruiseStatus': 'inactive',
 'destination': 'None',
 'frontLeftTirePressure': 35.0,
 'frontRightTirePressure': 35.0,
 'rearLeftTirePressure': 33.0,
 'rearRightTirePressure': 33.0,
 'long_context': False}

In [75]:
vars(output['states'][0]['environment']['VehicleControlAPI'])

{'_api_description': 'This tool belongs to the vehicle control system, which allows users to control various aspects of the car such as engine, doors, climate control, lights, and more.',
 '_random': <random.Random at 0x555cd6e4b2c0>,
 'fuelLevel': 15.5,
 'batteryVoltage': 12.8,
 'engine_state': 'stopped',
 'remainingUnlockedDoors': 0,
 'doorStatus': {'driver': 'locked',
  'passenger': 'locked',
  'rear_left': 'locked',
  'rear_right': 'locked'},
 'acTemperature': 22.0,
 'fanSpeed': 60,
 'acMode': 'auto',
 'humidityLevel': 45.0,
 'headLightStatus': 'off',
 'parkingBrakeStatus': 'released',
 '_parkingBrakeForce': 0.0,
 '_slopeAngle': 0.0,
 'brakePedalStatus': 'released',
 '_brakePedalForce': 0.0,
 'distanceToNextVehicle': 100.0,
 'cruiseStatus': 'inactive',
 'destination': 'None',
 'frontLeftTirePressure': 35.0,
 'frontRightTirePressure': 35.0,
 'rearLeftTirePressure': 33.0,
 'rearRightTirePressure': 33.0,
 'long_context': False}

In [74]:
vars(output['states'][0]['ground_truth_environment']['VehicleControlAPI'])

{'_api_description': 'This tool belongs to the vehicle control system, which allows users to control various aspects of the car such as engine, doors, climate control, lights, and more.',
 '_random': <random.Random at 0x555c0d67e7b0>,
 'fuelLevel': 15.5,
 'batteryVoltage': 12.8,
 'engine_state': 'stopped',
 'remainingUnlockedDoors': 4,
 'doorStatus': {'driver': 'unlocked',
  'passenger': 'unlocked',
  'rear_left': 'unlocked',
  'rear_right': 'unlocked'},
 'acTemperature': 22.0,
 'fanSpeed': 60,
 'acMode': 'auto',
 'humidityLevel': 45.0,
 'headLightStatus': 'off',
 'parkingBrakeStatus': 'released',
 '_parkingBrakeForce': 0.0,
 '_slopeAngle': 0.0,
 'brakePedalStatus': 'released',
 '_brakePedalForce': 0.0,
 'distanceToNextVehicle': 100.0,
 'cruiseStatus': 'inactive',
 'destination': 'None',
 'frontLeftTirePressure': 35.0,
 'frontRightTirePressure': 35.0,
 'rearLeftTirePressure': 33.0,
 'rearRightTirePressure': 33.0,
 'long_context': False}

In [28]:
print(trainer.tokenizer.decode(output["ids"][0]))

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


To check the status of all doors, I need to unlock and then lock each door. I'll start by unlocking the driver's door and checking if it's in the unlocked state. If it is, I'll lock it and repeat this process for the passenger door, rear left door, and rear right door. Once all doors are either unlocked or locked, I can conclude that all doors are properly locked. Now, I'll execute the door locking process using the `lock_doors` function.<|im_end|>


<module 'verifiers.rubrics.bfcl_inthinking_rubric' from '/workspace/verifiers/rubrics/bfcl_inthinking_rubric.py'>

In [29]:

importlib.reload(bfcl_inthinking_env)
env = bfcl_inthinking_env.BfclITEnv(
    dataset="bfcl",
    tools=[],
    max_num_turns=1,
    max_steps_per_turn=10,
    curriculum_learning=True,
    use_latest_trl=False,
)
trainer.env=env
trainer.env.tokenizer=trainer.tokenizer
ds_train = env.get_dataset(max_num_turns=1)
example=ds_train[0]
output = trainer.env.generate(
    prompts=[example["prompt"]],
    llm=trainer.llm,
    sampling_params=trainer.sampling_params,
    dataset_rows=[example],
    debug=True,  # True for detailed output
)

2025-04-20 13:44:31.974 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:281 - Initializing BfclITEnv (Single Tool Call Flow)
2025-04-20 13:44:31.975 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:315 - Initializing Scoring Rubric
2025-04-20 13:44:31.975 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:317 - Initializing LLM Parser
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
2025-04-20 13:44:31.975 | INFO     | verifiers.envs.bfcl_inthinking_env:get_dataset:324 - Preprocessing dataset bfcl for train split...


Map:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

2025-04-20 13:44:32.579 | INFO     | verifiers.envs.bfcl_inthinking_env:get_dataset:335 - Filtering train dataset where num_turns <= 1.


Filter:   0%|          | 0/379 [00:00<?, ? examples/s]

2025-04-20 13:44:32.596 | INFO     | verifiers.envs.bfcl_inthinking_env:generate:1181 - Using sampling params: stop=['</tool>', '<TASK_ERROR>', '<TASK_FINISHED>'], include_stop=True, max_tokens=2048


Initial User Prompt (State 0): You are an expert assistant that reasons step-by-step and uses tools to answer user questions.

# Available tools:
{"name": "add_contact", "description": "Add a contact to the workspace.", "parameters": {"type": "dict", "properties": {"user_name": {"type": "string", "description": "User name of contact to be added."}}, "required": ["user_name"]}, "response": {"type": "dict", "properties": {"added_status": {"type": "boolean", "description": "True if the contact was added successfully, False other...
Number of Rollouts: 1

--- Starting Generation Step 1 ---
---> Calling LLM for 1 states (add_generation_prompt=True)


2025-04-20 13:44:33.076 | WARNING  | verifiers.envs.bfcl_inthinking_env:step:659 - State 0: Last message is assistant but doesn't end with </tool_result>. Checking completion status.
2025-04-20 13:44:33.076 | ERROR    | verifiers.envs.bfcl_inthinking_env:step:665 - State 0: Ambiguous state - assistant message without result, but not completed. Marking error.
2025-04-20 13:44:33.076 | DEBUG    | verifiers.envs.bfcl_inthinking_env:cleanup_instances:1141 - Cleaning up 1 environment instance sets.


--- Processing 1 LLM responses ---
State 0 LLM Output Segment: '</think>
Sure, I can assist with that. First, let's check the status of the doors to ensure they are locked or unlocked. I will use the `lockDoors` tool for this purpose.
<tool>
{"name": "lockDoors", "args": {"unlock": false, "door": ["driver", "passenger", "rear_left", "rear_right"]}}
</tool>'
State 0: Detected </tool>
Attempting Tool Call: lockDoors({'unlock': False, 'door': ('driver', 'passenger', 'rear_left', 'rear_right')})
Found method lockDoors in class VehicleControlAPI
------------------------------ Step End (Processed 1 LLM responses) ------------------------------
--- Finished Generation Step 1 (All Completed: False) ---

--- Starting Generation Step 2 ---
---> Calling LLM for 1 states (continue_final_message=True)
--- Processing 1 LLM responses ---
State 0 LLM Output Segment: '<|im_end|>'
------------------------------ Step End (Processed 1 LLM responses) ------------------------------
--- Finished Generation S

In [14]:
print(output['messages'][0][0]['content'])

To check if all doors are correctly locked, we'll need to first verify that all doors are unlocked and then lock them if necessary. Let's use the 'lockDoors' tool to check the status of the doors.

<think>
We need to call the 'lockDoors' tool to check and confirm if all doors are properly locked. We'll also need to unlock and then re-lock the doors to ensure they are completely secure.
</think>

<tool> {"name": "lockDoors", "args": {"unlock": false, "door": ["driver", "passenger", "rear_left", "rear_right"]}} <tool_result>
{
  "lockStatus": "locked",
  "remainingUnlockedDoors": 0
}
</tool><tool_result>"Error decoding tool call JSON: Extra data: line 1 column 110 (char 109). Ensure the format is a single JSON object."</tool_result> Wait, 


In [13]:
print(trainer.tokenizer.decode(output["ids"][0]))

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


To check if all doors are correctly locked, we'll need to first verify that all doors are unlocked and then lock them if necessary. Let's use the 'lockDoors' tool to check the status of the doors.

<think>
We need to call the 'lockDoors' tool to check and confirm if all doors are properly locked. We'll also need to unlock and then re-lock the doors to ensure they are completely secure.
</think>

<tool> {"name": "lockDoors", "args": {"unlock": false, "door": ["driver", "passenger", "rear_left", "rear_right"]}} <tool_result>
{
  "lockStatus": "locked",
  "remainingUnlockedDoors": 0
}
</tool>

<tool_result>"Error decoding tool call JSON: Extra data: line 1 column 110 (char 109). Ensure the format is a single JSON object."</tool_result> Wait, 


In [13]:
output['messages']

[[{'role': 'assistant',
   'content': 'To verify and ensure all doors in your vehicle are properly locked, I will start by checking the status of the doors. This will allow me to determine whether each door is locked or not.\n\n</think>\n\nI will use the \'lockDoors\' tool to check the status of the doors.\n\n<tool name="lockDoors">{"unlock": false, "door": ["driver", "passenger", "rear_left", "rear_right"]}</tool><tool_result>["Error: Stray </tool> tag found."]</tool_result><|im_end|>'}]]

In [14]:
print(trainer.tokenizer.chat_template)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% set ns = namespace(is_first=false, is_tool=false, is_output_first=true, system_prompt='') %}{%- for message in messages %}{%- if message['role'] == 'system' %}{% set ns.system_prompt = message['content'] %}{%- endif %}{%- endfor %}{{bos_token}}{{ns.system_prompt}}{%- for message in messages %}{%- if message['role'] == 'user' %}{%- set ns.is_tool = false -%}{{'<｜User｜>' + message['content']}}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is none %}{%- set ns.is_tool = false -%}{%- for tool in message['tool_calls']%}{%- if not ns.is_first %}{{'<｜Assistant｜><｜tool▁calls▁begin｜><｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\n' + '```json' + '\n' + tool['function']['arguments'] + '\n' + '```' + '<｜tool▁call▁end｜>'}}{%- set ns.is_first = true -%}{%- else %}{{'\n' + '<｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['functio

In [16]:
len(example["prompt"][0])

2

In [18]:
example["prompt"][0]

{'content': 'You are an expert assistant that reasons step-by-step and uses tools to answer user questions.\n\n# Available tools:\n{"name": "add_contact", "description": "Add a contact to the workspace.", "parameters": {"type": "dict", "properties": {"user_name": {"type": "string", "description": "User name of contact to be added."}}, "required": ["user_name"]}, "response": {"type": "dict", "properties": {"added_status": {"type": "boolean", "description": "True if the contact was added successfully, False otherwise."}, "user_id": {"type": "string", "description": "User ID of the added contact."}, "message": {"type": "string", "description": "A message describing the result of the addition attempt."}}}}\n{"name": "delete_message", "description": "Delete the latest message sent to a receiver.", "parameters": {"type": "dict", "properties": {"receiver_id": {"type": "string", "description": "User ID of the user to send the message to."}, "message_id": {"type": "integer", "description": "ID 

In [49]:
len(output["mask"][0])

1722

In [51]:
masked = []
for x, y in zip(output["ids"][0], output["mask"][0]):
    if y == 1:
        masked.append(x)
    else:
        masked.append(0)
print(trainer.tokenizer.decode(masked))

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Let's break down this task step-by-step and use the available tools to make the booking.

<reasoning>
1. First, we need to contact customer support to get immediate support regarding the trip booking.
2. Create a ticket to track the booking and its issues.
3. Update the ticket with the required details for the booking.
4. Add the flight cost to the ticket.
5. Book the flight using the access token.
6. Wait for the booking to be made.
7. Once the booking is successful, notify customer support.
8. Edit the ticket to notify about the cancellation.
</reasoning>

<tool>
[{"name": "contact_customer_support", "args": {"booking_id": "ABC123", "message": "Need immediate support"}}, {"name": "book_flight", "args": {"access_token": "ABCD1234", "card_id": "id_1234", "travel_date": "2024-12-15", "travel_from": "JFK", "travel_to": "LAX", "travel_class": "economy", "travel_cost": 4500}}, {"name": "edit_ticket", "args": {"ticket_id": "ABC123", "updates": {"status": "Pending Ticket", "description": "Fl

In [45]:
trainer.tokenizer.decode([1, 1, 1, 1])

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


'""""'

In [40]:
output["ids"]

[[10061,
  594,
  1438,
  1495,
  419,
  3383,
  3019,
  14319,
  29208,
  323,
  990,
  279,
  2500,
  7375,
  311,
  1281,
  279,
  21857,
  382,
  27,
  19895,
  287,
  397,
  16,
  13,
  5512,
  11,
  582,
  1184,
  311,
  3645,
  6002,
  1824,
  311,
  633,
  13922,
  1824,
  8826,
  279,
  8411,
  21857,
  624,
  17,
  13,
  4230,
  264,
  11727,
  311,
  3754,
  279,
  21857,
  323,
  1181,
  4714,
  624,
  18,
  13,
  5549,
  279,
  11727,
  448,
  279,
  2567,
  3565,
  369,
  279,
  21857,
  624,
  19,
  13,
  2691,
  279,
  10971,
  2783,
  311,
  279,
  11727,
  624,
  20,
  13,
  5893,
  279,
  10971,
  1667,
  279,
  2615,
  3950,
  624,
  21,
  13,
  13824,
  369,
  279,
  21857,
  311,
  387,
  1865,
  624,
  22,
  13,
  9646,
  279,
  21857,
  374,
  6849,
  11,
  15456,
  6002,
  1824,
  624,
  23,
  13,
  8340,
  279,
  11727,
  311,
  15456,
  911,
  279,
  35835,
  624,
  522,
  19895,
  287,
  1339,
  27,
  14172,
  397,
  58,
  4913,
  606,
  788,
  330,
  6287,


In [17]:
state = output["states"][0]

In [18]:
state

{'messages': [{'content': 'You are an expert in composing functions. You are given a question from a user and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to complete the task.\nYou have access to the following tools to help solve the task:\n\n{"name": "close_ticket", "description": "Close a ticket.", "parameters": {"type": "dict", "properties": {"ticket_id": {"type": "integer", "description": "ID of the ticket to be closed. "}}, "required": ["ticket_id"]}, "response": {"type": "dict", "properties": {"status": {"type": "string", "description": "Status of the close operation."}}}}\n{"name": "create_ticket", "description": "Create a ticket in the system and queue it.", "parameters": {"type": "dict", "properties": {"title": {"type": "string", "description": "Title of the ticket."}, "description": {"type": "string", "description": "Description of the ticket. Defaults to an empty string.", "default": ""}, "priority": {"type": "int

In [57]:
num_func_matches = 0
num_func_total = 0
model_func_calls = state["successful_func_calls"]
ground_truth_func_calls = json.loads(state["dataset_row"]["answer"])
assert len(model_func_calls) == len(ground_truth_func_calls)

for model_calls, gt_calls_str in zip(model_func_calls, ground_truth_func_calls):
    gt_calls = [
        trainer.env.rubric._parse_function_call(call_str) for call_str in gt_calls_str
    ]

    def make_hashable(value):
        if isinstance(value, dict):
            return frozenset((k, make_hashable(v)) for k, v in value.items())
        elif isinstance(value, list):
            return tuple(make_hashable(item) for item in value)
        elif isinstance(value, set):
            return frozenset(make_hashable(item) for item in value)
        elif isinstance(value, tuple):
            return tuple(make_hashable(item) for item in value)
        try:
            hash(value)
            return value
        except TypeError:
            return str(value)

    comparable_model_calls = [
        (
            call["name"],
            frozenset((k, make_hashable(v)) for k, v in call["args"].items()),
        )
        for call in model_calls
    ]

    for call in gt_calls:
        if "args" in call:
            for key, value in call["args"].items():
                if isinstance(value, list):
                    call["args"][key] = tuple(value)
        else:
            raise Exception(
                "Error in Parsing Ground Truth Function Call is Not Expected!!"
            )

    comparable_gt_calls = [
        (
            call["name"],
            frozenset((k, make_hashable(v)) for k, v in call["args"].items()),
        )
        for call in gt_calls
    ]
    print(f"Comparable Model Calls: {comparable_model_calls}")
    print(f"Comparable Ground Truth Calls: {comparable_gt_calls}")

    is_match, _ = trainer.env.rubric._is_subsequence_unordered(
        comparable_gt_calls, comparable_model_calls
    )
    print(f"Is Subsequence: {is_match}")
    num_func_matches += int(is_match)
    num_func_total += 1
func_score = num_func_matches / num_func_total
print(
    f"Function Call Match Score: {func_score}, num_func_matches: {num_func_matches}, num_func_total: {num_func_total}"
)

Comparable Model Calls: [('book_flight', frozenset({('travel_cost', 4500), ('card_id', 'id_1234'), ('travel_to', 'LAX'), ('travel_class', 'economy'), ('travel_date', '2024-12-15'), ('access_token', 'ABCD1234'), ('travel_from', 'JFK')}))]
Comparable Ground Truth Calls: [('book_flight', frozenset({('travel_class', 'business'), ('travel_cost', 4500.0), ('card_id', 'id_1234'), ('travel_to', 'LAX'), ('travel_date', '2024-12-15'), ('access_token', 'ABCD1234'), ('travel_from', 'JFK')})), ('cancel_booking', frozenset({('booking_id', '3426812'), ('access_token', 'ABCD1234')}))]
Is Subsequence: False
Function Call Match Score: 0.0, num_func_matches: 0, num_func_total: 1
